In [0]:
import pandas as pd
import random
import string
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType,
    FloatType, DoubleType, BooleanType, DateType, TimestampType, ArrayType
)
# Initialize SparkSession
# spark = SparkSession.builder \
#     .appName("ExcelMetadataWorkflow") \
#     .getOrCreate()

In [0]:
!pip install openpyxl

     |████████████████████████████████| 250 kB 6.3 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4837e831-749c-4365-82ae-f7091d969838/bin/python -m pip install --upgrade pip' command.


In [0]:
# -------------------------------------
# Step 1. Read all sheets from the Excel file.
# -------------------------------------
excel_path = "/tmp/HPE_NVDA_datagen.xlsx"  # update this path

# Read every sheet into a dictionary: keys are sheet names, values are DataFrames.
sheets = pd.read_excel(excel_path, sheet_name=None)
# sheets = spark.read.  
sheet_names = list(sheets.keys())
print("Found sheets:", sheet_names)

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File <command-2280788391158464>:7
      4 excel_path = "/tmp/HPE_NVDA_datagen.xlsx"  # update this path
      6 # Read every sheet into a dictionary: keys are sheet names, values are DataFrames.
----> 7 sheets = pd.read_excel(excel_path, sheet_name=None)
      8 # sheets = spark.read.  
      9 sheet_names = list(sheets.keys())

File /databricks/python/lib/python3.9/site-packages/pandas/util/_decorators.py:311, in deprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper(*args, **kwargs)
    305 if len(args) > num_allow_args:
    306     warnings.warn(
    307         msg.format(arguments=arguments),
    308         FutureWarning,
    309         stacklevel=stacklevel,
    310     )
--> 311 return func(*args, **kwargs)

File /databricks/python/lib/python3.9/site-packages/pandas/io/excel/_base.py:457, in read_excel(io, sheet_name

In [0]:
# -------------------------------------
# Step 2. Process the tables overview (first sheet)
# -------------------------------------
# Assumption: The first sheet (e.g. "Tables") lists the table names and approximate row counts.
tables_overview_df = sheets[sheet_names[0]]
# Adjust these column names if your Excel file uses different names.
table_names = tables_overview_df["masked_table_id"].tolist()
approx_row_counts = tables_overview_df["num_rows_approx"].tolist()

print("Tables and approximate row counts:")
for tbl, cnt in zip(table_names, approx_row_counts):
    print(f"  {tbl}: ~{cnt} rows")


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2280788391158465>:5
      1 # -------------------------------------
      2 # Step 2. Process the tables overview (first sheet)
      3 # -------------------------------------
      4 # Assumption: The first sheet (e.g. "Tables") lists the table names and approximate row counts.
----> 5 tables_overview_df = sheets[sheet_names[0]]
      6 # Adjust these column names if your Excel file uses different names.
      7 table_names = tables_overview_df["masked_table_id"].tolist()

NameError: name 'sheets' is not defined

In [0]:
# -------------------------------------
# Step 3. Read each table's metadata (columns, types, etc.)
# -------------------------------------
# Here we assume that the sheet name for each table is the same as the table name.
table_metadata = {}
for tbl in table_names:
    if tbl in sheets:
        meta_df = sheets[tbl]
        table_metadata[tbl] = meta_df
        print(f"Loaded metadata for table '{tbl}'.")
    else:
        print(f"Warning: No metadata sheet found for table '{tbl}'.")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2280788391158466>:6
      1 # -------------------------------------
      2 # Step 3. Read each table's metadata (columns, types, etc.)
      3 # -------------------------------------
      4 # Here we assume that the sheet name for each table is the same as the table name.
      5 table_metadata = {}
----> 6 for tbl in table_names:
      7     if tbl in sheets:
      8         meta_df = sheets[tbl]

NameError: name 'table_names' is not defined

In [0]:
# -------------------------------------
# Step 4. Define a mapping from your Excel type names to Spark types.
# -------------------------------------
spark_type_mapping = {
    "StringType()": StringType(),
    "StringType": StringType(),
    "IntegerType()": IntegerType(),
    "IntegerType()": IntegerType(),
    "LongType()": LongType(),
    "FloatType()": FloatType(),
    "DoubleType()": DoubleType(),
    "BooleanType()": BooleanType(),
    "BooleanType()": BooleanType(),
    "DateType()": DateType(),
    "TimestampType()": TimestampType(),
    "ArrayType(IntegerType(), True)": ArrayType(IntegerType(), True),
    "ArrayType(StringType(), True)": ArrayType(StringType(), True)
}

def create_schema(meta_df):
    """
    Create a Spark schema (StructType) from the metadata DataFrame.
    For numerical types, if "min" and "max" are provided, they are stored in the field metadata.
    This version ensures that the type from the spreadsheet is used (if it matches).
    """
    fields = []
    # Ensure that the range columns exist in the DataFrame.
    has_range = ("min" in meta_df.columns) and ("max" in meta_df.columns)
    
    for idx, row in meta_df.iterrows():
        col_name = row["masked_column_name"]
        # Convert the Type from the spreadsheet to a lower-case string.
        type_str = str(row["spark_data_type"]).strip() if pd.notna(row["spark_data_type"]) else "string"
        spark_type = spark_type_mapping.get(type_str)
        
        if spark_type is None:
            # If the type is not recognized, warn and default to StringType.
            print(f"Warning: Unrecognized type '{row['spark_data_type']}' for column '{col_name}'. Using StringType.")
            spark_type = StringType()
        
        md = {}
        # For numerical types, if min and max values are provided, store them in metadata.
        if isinstance(spark_type, (IntegerType, LongType, FloatType, DoubleType)) and has_range:
            if pd.notna(row["min"]) and pd.notna(row["max"]):
                md["min"] = row["min"]
                md["max"] = row["max"]
        
        fields.append(StructField(col_name, spark_type, True, metadata=md))
    
    return StructType(fields)

# Create a dictionary of schemas for each table.
schemas = {}
for tbl, meta_df in table_metadata.items():
    schema = create_schema(meta_df)
    schemas[tbl] = schema
    print(f"Schema for table '{tbl}': {schema}")

In [0]:
# -------------------------------------
# Step 5. Process join information.
# -------------------------------------
# Assumption: The final sheet (last sheet) is named "Joins" and holds the join definitions.
join_info_df = sheets[sheet_names[1]]
joins = []
# Here we assume join_info_df has columns: "LeftTable", "LeftColumn", "RightTable", "RightColumn", and optionally "JoinType"
for idx, row in join_info_df.iterrows():
    join_detail = {
        "left_table": row["table1"],
        "right_table": row["table2"],
        "join_method": row["join_method"],
        "left_column": row["column1"],
        "right_column": row["column2"]
    }
    joins.append(join_detail)

print("Join definitions:")
for join in joins:
    print(f"  {join['left_table']}.{join['left_column']} {join['join_method'].upper()} JOIN {join['right_table']}.{join['right_column']}")


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2280788391158468>:5
      1 # -------------------------------------
      2 # Step 5. Process join information.
      3 # -------------------------------------
      4 # Assumption: The final sheet (last sheet) is named "Joins" and holds the join definitions.
----> 5 join_info_df = sheets[sheet_names[1]]
      6 joins = []
      7 # Here we assume join_info_df has columns: "LeftTable", "LeftColumn", "RightTable", "RightColumn", and optionally "JoinType"

NameError: name 'sheets' is not defined

In [0]:
# -------------------------------------
# (Optional) Step 6. Build a dynamic join query.
# -------------------------------------
# If you later load your data into Spark DataFrames and register them as temporary views,
# you could build and execute a join query dynamically. For example, suppose you have:
#    df_customers = spark.read.csv("customers.csv", schema=schemas["customers"], header=True)
#    df_customers.createOrReplaceTempView("customers")
#    ... and similarly for other tables.
#
# The code below builds a join SQL string assuming sequential joining.
if table_names:
    join_query = f"SELECT * FROM {table_names[0]}"
    for join in joins:
        # Note: This simple logic assumes that the join order is appropriate.
        join_query += (
            f" {join['join_method'].upper()} JOIN {join['right_table']} "
            f"ON {join['left_table']}.{join['left_column']} = {join['right_table']}.{join['right_column']}"
        )
    print("Constructed join query:")
    print(join_query)
    # To execute the query once the tables are registered as temp views:
    # result_df = spark.sql(join_query)
    # result_df.show()

# -------------------------------------
# Now you have:
#  - 'table_names' and 'approx_row_counts' from the overview.
#  - 'table_metadata': a dictionary mapping table names to their metadata DataFrames.
#  - 'schemas': a dictionary mapping table names to Spark schemas.
#  - 'joins': a list of dictionaries describing the join relationships.
#
# You can now use this information to drive your Spark ETL/processing workflow.
#
# When finished, stop the Spark session (if running in a script).
# spark.stop()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2280788391158469>:11
      1 # -------------------------------------
      2 # (Optional) Step 6. Build a dynamic join query.
      3 # -------------------------------------
   (...)
      9 #
     10 # The code below builds a join SQL string assuming sequential joining.
---> 11 if table_names:
     12     join_query = f"SELECT * FROM {table_names[0]}"
     13     for join in joins:
     14         # Note: This simple logic assumes that the join order is appropriate.

NameError: name 'table_names' is not defined

# Actually Generate Data

In [0]:
# ========================================
# PART 2: Generate random data for each table and register as temp views
# ========================================

def generate_random_dataframe(schema, num_rows):
    """
    Given a Spark StructType schema and a number of rows, generate a DataFrame with random data
    using Spark’s distributed operations.
    For numerical types, if metadata has "min" and "max", those bounds are used.
    """
    # Start with a DataFrame with a column "id" (this DataFrame is generated in a distributed fashion)
    df = spark.range(num_rows)
    
    # For each field in the schema, add a column with a random value.
    for field in schema.fields:
        col_name = field.name
        dt = field.dataType
        md = field.metadata or {}
        
        if isinstance(dt, (IntegerType, LongType)):
            # Use provided min and max if available; otherwise default to 1 and 1000.
            min_val = md.get("min", 1)
            max_val = md.get("max", 1000)
            expr = (F.rand() * (float(max_val) - float(min_val)) + float(min_val))
            # Cast appropriately.
            if isinstance(dt, IntegerType):
                df = df.withColumn(col_name, expr.cast("int"))
            else:
                df = df.withColumn(col_name, expr.cast("long"))
                
        elif isinstance(dt, (FloatType, DoubleType)):
            min_val = md.get("min", 0.0)
            max_val = md.get("max", 1000.0)
            expr = (F.rand() * (float(max_val) - float(min_val)) + float(min_val))
            if isinstance(dt, FloatType):
                df = df.withColumn(col_name, expr.cast("float"))
            else:
                df = df.withColumn(col_name, expr.cast("double"))
                
        elif isinstance(dt, BooleanType):
            # Generate a boolean value based on a threshold.
            df = df.withColumn(col_name, F.rand() > 0.5)
            
        elif isinstance(dt, DateType):
            # Generate a random date by adding a random number of days (e.g., 0 to 9000) to a base date.
            df = df.withColumn(col_name, F.expr("date_add('2000-01-01', cast(rand() * 9000 as int))"))
            
        elif isinstance(dt, TimestampType):
            # Generate a random timestamp by first generating a random date and then converting it.
            df = df.withColumn(col_name, F.expr("to_timestamp(date_add('2000-01-01', cast(rand() * 9000 as int)))"))
            
        elif isinstance(dt, StringType):
            # Use the built-in uuid() function for random strings.
            df = df.withColumn(col_name, F.expr("uuid()"))
            
        else:
            # For any unrecognized type, set the column to null.
            df = df.withColumn(col_name, F.lit(None))
            
    # Drop the original "id" column.
    return df.drop("id")

# Create and register a DataFrame for each table using the distributed random data generation.
# NOTE: THIS WAS SCALED DOWN FOR TESTING PURPOSES. UNCOMMENT LINE 74 AND COMMENT OUT LINES 68-73 FOR REAL TESTING
dfs = {}
for tbl, count in zip(table_names, approx_row_counts):
    schema = schemas[tbl]
    if tbl == 'table_a':
        num_rows = 100000000
    elif tbl == 'table_c':
        num_rows = 21000000
    else:
        num_rows = int(count)
    # num_rows = int(count)
    df = generate_random_dataframe(schema, num_rows)
    dfs[tbl] = df
    print(f"Created DataFrame for table '{tbl}' with {num_rows} random rows.")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2280788391158471>:66
     63 # Create and register a DataFrame for each table using the distributed random data generation.
     64 # NOTE: THIS WAS SCALED DOWN FOR TESTING PURPOSES. UNCOMMENT LINE 74 AND COMMENT OUT LINES 68-73 FOR REAL TESTING
     65 dfs = {}
---> 66 for tbl, count in zip(table_names, approx_row_counts):
     67     schema = schemas[tbl]
     68     if tbl == 'table_a':

NameError: name 'table_names' is not defined

# GroupBys

In [0]:
table_a = dfs['table_a']
gb_test1 = table_a.groupBy(['col_a_1', 'col_a_3', 'col_a_5', 'col_a_7', 'col_a_9']).count()

gb_test1.write.format("noop").mode("overwrite").save()

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-2280788391158473>:1
----> 1 table_a = dfs['table_a']
      2 gb_test1 = table_a.groupBy(['col_a_1', 'col_a_3', 'col_a_5', 'col_a_7', 'col_a_9']).count()
      4 gb_test1.write.format("noop").mode("overwrite").save()

KeyError: 'table_a'

In [0]:
table_a = dfs['table_a']
gb_test2 = table_a.groupBy(['col_a_1']).count()

gb_test2.write.format("noop").mode("overwrite").save()

# Joins

### table_c `col_1` and `col_3` are both ArrayType(IntegerType) columns in real life, most of the time the array is one value long, but sometimes it's multiple values. Try and figure this out at home.

In [0]:
table_a = dfs['table_a']
table_b = dfs['table_b']

join_test1 = table_a.join(table_b, [
    table_a["col_a_1"]==table_b["col_b_8"],
    table_a["col_a_3"]==table_b["col_b_3"],
    table_a["col_a_5"]==table_b["col_b_9"],
    table_a["col_a_7"]==table_b["col_b_1"],
],
how='left')
join_test1.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_c = dfs['table_c']

join_test2 = table_a.join(table_c, [
    table_a["col_a_1"]==table_c["col_c_10"],
    table_a["col_a_3"]==table_c["col_c_9"],
    table_a["col_a_9"]==table_c["col_c_11"],
],
how='left')

join_test2.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_d = dfs['table_d']

join_test3 = table_a.join(table_d, [
    table_a["col_a_1"]==table_d["col_d_0"],
    table_a["col_a_5"]==table_d["col_d_1"],
],
how='left')

join_test3.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_e = dfs['table_e']

join_test4 = table_a.join(table_e, table_a["col_a_1"]==table_e["col_e_0"], how='left')

join_test4.write.format("noop").mode("overwrite").save()

In [0]:
table_c = dfs['table_c']
table_e = dfs['table_e']

join_test5 = table_c.join(table_e, table_c["col_c_5"]==table_e["col_e_0"], how='left')

join_test5.write.format("noop").mode("overwrite").save()

In [0]:
table_c = dfs['table_c']
table_e = dfs['table_e']

join_test6 = table_c.join(table_e, table_c["col_c_10"]==table_e["col_e_0"], how='left')

join_test6.write.format("noop").mode("overwrite").save()

In [0]:
linked_join_test = (
    table_a
    .join(
        table_b,
        [
            table_a["col_a_1"] == table_b["col_b_8"],
            table_a["col_a_3"] == table_b["col_b_3"],
            table_a["col_a_5"] == table_b["col_b_9"],
            table_a["col_a_7"] == table_b["col_b_1"],
        ],
        how="left"
    )
    .join(
        table_c,
        [
            table_a["col_a_1"] == table_c["col_c_10"],
            table_a["col_a_3"] == table_c["col_c_9"],
            table_a["col_a_9"] == table_c["col_c_11"],
        ],
        how="left"
    )
    .join(
        table_d,
        [
            table_a["col_a_1"] == table_d["col_d_0"],
            table_a["col_a_5"] == table_d["col_d_1"],
        ],
        how="left"
    )
    .join(
        table_e,
        table_a["col_a_1"] == table_e["col_e_0"],
        how="left"
    )
)

linked_join_test.write.format("noop").mode("overwrite").save()

# Breadth First Search

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create or get your Spark session
# spark = SparkSession.builder.getOrCreate()

# Assume your input DataFrame 'df' has columns "col_0" and "col_1".
# We create an 'edges' DataFrame with "src" and "dst" columns.
df = dfs['table_a'].limit(100000)

edges = df.select(F.col("col_a_3").alias("src"), F.col("col_a_7").alias("dst"))

# Define the BFS starting point.
# Change the 'source' variable to the vertex from which you want to start the BFS.
source = 1000  # For example, use "A" as the starting vertex

# Create the initial frontier: the source vertex with distance 0.
frontier = spark.createDataFrame([(source, 0)], ["vertex", "distance"])

# Create a DataFrame to keep track of all visited vertices (and their distance from the source).
visited = frontier

# Loop until there are no new nodes to visit.
while frontier.count() > 0:
    # 1. Find neighbors: join the current frontier with the edges DataFrame.
    #    Each neighbor gets a distance equal to (current distance + 1).
    new_neighbors = frontier.join(edges, frontier.vertex == edges.src) \
                            .select(edges.dst.alias("vertex"),
                                    (frontier.distance + 1).alias("distance"))
    
    # 2. Exclude vertices that have already been visited.
    new_neighbors = new_neighbors.join(visited, on="vertex", how="left_anti").distinct()
    
    # 3. If no new vertices are found, exit the loop.
    if new_neighbors.count() == 0:
        break
    
    # 4. Add the new neighbors to the visited set.
    visited = visited.union(new_neighbors).distinct()
    
    # 5. Update the frontier to be the new neighbors.
    frontier = new_neighbors

# The 'visited' DataFrame now contains all vertices reachable from the source,
# along with the minimum number of steps (distance) from the source.
visited.write.format("noop").mode("overwrite").save()

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-2697433332302766>:9
      2 from pyspark.sql import functions as F
      4 # Create or get your Spark session
      5 # spark = SparkSession.builder.getOrCreate()
      6 
      7 # Assume your input DataFrame 'df' has columns "col_0" and "col_1".
      8 # We create an 'edges' DataFrame with "src" and "dst" columns.
----> 9 df = dfs['table_a'].limit(100000)
     11 edges = df.select(F.col("col_a_3").alias("src"), F.col("col_a_7").alias("dst"))
     13 # Define the BFS starting point.
     14 # Change the 'source' variable to the vertex from which you want to start the BFS.

KeyError: 'table_a'

# PageRank

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create or get your Spark session
# spark = SparkSession.builder.getOrCreate()

table_a = dfs['table_a']
df = table_a.limit(10000) # Change as needed

# Set the reset (teleportation) probability and the number of iterations
alpha = 0.15
maxIter = 10

# Choose the personalized seed: take the first value from col_a_3
seed = df.select("col_a_3").first()[0]

# 1. Create the vertices DataFrame: union of unique IDs from col_a_3 and col_a_7.
vertices = (
    df.select(F.col("col_a_3").alias("id"))
      .union(df.select(F.col("col_a_7").alias("id")))
      .distinct()
)

# 2. Create the edges DataFrame: define edge from col_a_3 to col_a_7.
edges = df.select(F.col("col_a_3").alias("src"), F.col("col_a_7").alias("dst"))

# 3. Compute out-degrees: count of outgoing edges for each source vertex.
out_degrees = edges.groupBy("src").agg(F.count("*").alias("out_degree"))

# 4. Initialize each vertex with a PageRank value:
#    The seed gets 1.0 and all others start with 0.0.
vertices_rank = vertices.withColumn(
    "rank", F.when(F.col("id") == seed, 1.0).otherwise(0.0)
)

# 5. Iteratively update the PageRank values.
for i in range(maxIter):
    # 5a. For each edge, compute the contribution from its source.
    #     Join the edges with the current vertex ranks and the out-degrees.
    contribs = (
        edges.join(vertices_rank, edges.src == vertices_rank.id)
             .join(out_degrees, edges.src == out_degrees.src)
             .select(
                 edges.dst.alias("id"),
                 (vertices_rank.rank / out_degrees.out_degree).alias("contrib")
             )
    )
    
    # 5b. Sum the contributions arriving at each vertex.
    contribs_sum = contribs.groupBy("id").agg(F.sum("contrib").alias("sum_contrib"))
    
    # 5c. Compute the total rank from dangling nodes (vertices with no outgoing edges).
    dangling = (
        vertices_rank.join(out_degrees, vertices_rank.id == out_degrees.src, "left")
                     .withColumn("out_degree", F.coalesce(F.col("out_degree"), F.lit(0)))
                     .where(F.col("out_degree") == 0)
    )
    dangling_sum = dangling.agg(F.sum("rank").alias("dangling_sum")).collect()[0]["dangling_sum"]
    if dangling_sum is None:
        dangling_sum = 0.0
    
    # 5d. Update each vertex's rank:
    #     - If the vertex is the seed, it gets the reset term (alpha)
    #       plus (1 - alpha) times (its incoming contributions plus dangling rank).
    #     - Otherwise, it just gets (1 - alpha) times its incoming contributions.
    vertices_rank = (
        vertices.join(contribs_sum, on="id", how="left")
                .na.fill({"sum_contrib": 0.0})
                .withColumn("rank", 
                    F.when(F.col("id") == seed,
                           alpha + (1 - alpha) * (F.col("sum_contrib") + dangling_sum)
                    ).otherwise(
                           (1 - alpha) * F.col("sum_contrib")
                    )
                )
    )

# 6. Display the final personalized PageRank values.
vertices_rank.show()


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-2697433332302768>:7
      2 from pyspark.sql import functions as F
      4 # Create or get your Spark session
      5 # spark = SparkSession.builder.getOrCreate()
----> 7 table_a = dfs['table_a']
      8 df = table_a.limit(10000) # Change as needed
     10 # Set the reset (teleportation) probability and the number of iterations

KeyError: 'table_a'

# Barabasi-Albert Graph

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import random

# def generate_ip(node_id):
def generate_nodes(node_id):
    return f"192.168.{node_id // 256}.{node_id % 256}"

# def generate_base_nodes(node_id):
def generate_five_t(node_id):
    col_0 = generate_nodes(src)
    col_1 = generate_nodes(dst)
    col_2 = random.randint(1000, 70000)
    col_3 = random.randint(1000, 70000)
    col_4 = random.choice(["AAA", "BBB"])
    return (col_0, col_1, col_2, col_3, col_4)

def generate_ba_graph(total_nodes, m):
    """
    Generate a graph using the Barabási–Albert (BA) model.
    - total_nodes: Total number of nodes in the graph.
    - m: Number of edges each new node will attach to.
    
    Returns a list of edges.
    Each edge is represented as a tuple:
      (src_node, dst_node, col_0, col_1, col_2, col_3, col_4)
    """
    edges = []
    # Step 1: Create an initial complete graph among the first m nodes.
    initial_nodes = list(range(m))
    for i in range(m):
        for j in range(i + 1, m):
            five_t = generate_five_t(i, j)
            edges.append((i, j,
                          five_t[0], five_t[1],
                          five_t[2], five_t[3],
                          five_t[4]))
    
    # Step 2: Maintain a list of nodes repeated by their degree.
    # This list will be used for preferential attachment.
    repeated_nodes = []
    # In the complete graph, each node has degree (m - 1)
    for node in initial_nodes:
        repeated_nodes.extend([node] * (m - 1))
    
    # Step 3: Iteratively add new nodes to the graph.
    for new_node in range(m, total_nodes):
        targets = set()
        # Sample m unique target nodes with probability proportional to degree.
        while len(targets) < m:
            target = random.choice(repeated_nodes)
            targets.add(target)
        for target in targets:
            five_tuple = generate_five_tuple(new_node, target)
            edges.append((new_node, target,
                          five_tuple[0], five_tuple[1],
                          five_tuple[2], five_tuple[3],
                          five_tuple[4]))
            # Update the repeated_nodes list:
            repeated_nodes.append(new_node)
            repeated_nodes.append(target)
    return edges

if __name__ == "__main__":

    # Parameters for BA graph
    total_nodes = 1_000_000   # Adjust as needed
    m = 3                # Each new node connects to m existing nodes

    # Generate the BA graph edges with five-tuple data
    edges = generate_ba_graph(total_nodes, m)

    # Define a schema for the edges DataFrame
    schema = StructType([
        StructField("src_node", IntegerType(), False),
        StructField("dst_node", IntegerType(), False),
        StructField("col_0", StringType(), False),
        StructField("col_1", StringType(), False),
        StructField("col_2", IntegerType(), False),
        StructField("col_3", IntegerType(), False),
        StructField("col_4", StringType(), False)
    ])

    # Create a DataFrame from the edge list
    edges_df = spark.createDataFrame(edges, schema)
    # edges_df.cache()

    # Show a few rows of the generated graph
    # edges_df.show(10, truncate=False)
    edges_df.write.format("noop").mode("overwrite").save()

    # Optionally, write the graph data to disk (e.g., CSV)
    # edges_df.write.csv("ba_graph_output.csv", header=True)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-2280788391158487>:71
     68 m = 3                # Each new node connects to m existing nodes
     70 # Generate the BA graph edges with five-tuple data
---> 71 edges = generate_ba_graph(total_nodes, m)
     73 # Define a schema for the edges DataFrame
     74 schema = StructType([
     75     StructField("src_node", IntegerType(), False),
     76     StructField("dst_node", IntegerType(), False),
   (...)
     81     StructField("col_4", StringType(), False)
     82 ])

File <command-2280788391158487>:33, in generate_ba_graph(total_nodes, m)
     31 for i in range(m):
     32     for j in range(i + 1, m):
---> 33         five_t = generate_five_t(i, j)
     34         edges.append((i, j,
     35                       five_t[0], five_t[1],
     36                       five_t[2], five_t[3],
     37                       fi